In [ ]:
from typing import TYPE_CHECKING, Any, Generic, TypeVar

if TYPE_CHECKING:
    import pandas as pd

A = TypeVar('A')

class LazyChunk(Generic[A]):
    def __call__(self, columns: list[str] | None = None, chunksize: int | None = None) -> A:
        raise NotImplementedError()

class LazyDataset(Generic[A]):
    def __init__(self, merged_load: LazyChunk[A], partitions: dict[str, LazyChunk[A]] | None = None) -> None:
        self._partitions = partitions
        self.merged_load = merged_load

    def __call__(self, columns: list[str] | None = None) -> Any:
        return self.merged_load(columns=columns)

    def __getitem__(self, pid: str) -> LazyChunk[A]:
        assert self._partitions, "No partitions found."
        return self._partitions[pid]
    
    def __iter__(self):
        if self._partitions:
            return iter(self._partitions)
        return iter([self.merged_load])

class LazyFrame(LazyDataset["pd.DataFrame"]):
    ...